In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

In [39]:
# Load the data
churnData = pd.read_csv('files_for_lab/Customer-Churn.csv')

In [41]:
# Display data
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [43]:
# Check data types
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [45]:
# Data preprocessing and model building

In [47]:
# Convert 'TotalCharges' to numeric
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'], errors='coerce')

In [49]:
# Handle missing values and fill with median
churnData['TotalCharges'].fillna(churnData['TotalCharges'].median(), inplace=True)

/var/folders/0b/0nt_hmld68j1x9vbn6980snc0000gn/T/ipykernel_45628/3592667950.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  churnData['TotalCharges'].fillna(churnData['TotalCharges'].median(), inplace=True)


In [51]:
# Encoding categorical values
churnData = pd.get_dummies(churnData, columns=[
    'gender', 'Partner', 'Dependents', 'PhoneService', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract'], drop_first=True)

In [53]:
# Convert 'Churn' to a binary column with 1 indicating churn 'yes' and 0 indicating no churn 'no'
churnData['Churn'] = churnData['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

In [55]:
churnData.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,OnlineSecurity_No internet service,...,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year
0,0,1,29.85,29.85,0,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0,34,56.95,1889.50,0,True,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
2,0,2,53.85,108.15,1,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,0,45,42.30,1840.75,0,True,False,False,False,False,...,False,True,False,True,False,False,False,False,True,False
4,0,2,70.70,151.65,1,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [57]:
# Feature selection
X = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]
y = churnData['Churn']

In [59]:
# Standarization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [61]:
# Split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [63]:
# Train logistic regression model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [65]:
# Evaluate model
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test data (imbalanced):", accuracy)

Accuracy on test data (imbalanced): 0.7931850449597728


In [67]:
# Check class imbalance
print("Class distribution in target variable:")
print(y.value_counts())

Class distribution in target variable:
Churn
0    5174
1    1869
Name: count, dtype: int64


In [69]:
# Upsampling and downsampling to address imbalance
from sklearn.utils import resample

# Separate majority and minority classes
churn_majority = churnData[churnData.Churn == 0]
churn_minority = churnData[churnData.Churn == 1]

# Upsample minority class
churn_minority_upsampled = resample(churn_minority, replace=True, n_samples=len(churn_majority), random_state=42)
churn_upsampled = pd.concat([churn_majority, churn_minority_upsampled])

# Downsample majority class
churn_majority_downsampled = resample(churn_majority, replace=False, n_samples=len(churn_minority), random_state=42)
churn_downsampled = pd.concat([churn_majority_downsampled, churn_minority])

In [89]:
# Evaluate models on resampled data
from sklearn.metrics import precision_score, recall_score, f1_score

In [91]:
# Original feature selection and target
X = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]
y = churnData['Churn']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Stratified split for the original imbalanced data for baseline comparison
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# Function to train and evaluate a logistic regression model
def train_evaluate_model(X_train, y_train, X_test, y_test, dataset_type):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"{dataset_type} Data - Model Evaluation:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}\n")
    
    return accuracy, precision, recall, f1

In [93]:
# Baseline Evaluation (Original Imbalanced Data)
train_evaluate_model(X_train_orig, y_train_orig, X_test_orig, y_test_orig, "Original Imbalanced")

# Upsample the minority class
churn_majority = churnData[churnData.Churn == 0]
churn_minority = churnData[churnData.Churn == 1]

churn_minority_upsampled = resample(churn_minority, replace=True, n_samples=len(churn_majority), random_state=42)
churn_upsampled = pd.concat([churn_majority, churn_minority_upsampled])

X_upsampled = churn_upsampled[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]
y_upsampled = churn_upsampled['Churn']

# Scale and split the upsampled data
X_upsampled_scaled = scaler.fit_transform(X_upsampled)
X_train_up, X_test_up, y_train_up, y_test_up = train_test_split(X_upsampled_scaled, y_upsampled, test_size=0.3, random_state=42)

Original Imbalanced Data - Model Evaluation:
Accuracy: 0.7913
Precision: 0.6538
Recall: 0.4545
F1 Score: 0.5363



In [95]:
# Evaluate model on upsampled data
train_evaluate_model(X_train_up, y_train_up, X_test_up, y_test_up, "Upsampled")

Upsampled Data - Model Evaluation:
Accuracy: 0.7308
Precision: 0.7183
Recall: 0.7354
F1 Score: 0.7268



(0.730756843800322, 0.7183462532299741, 0.7354497354497355, 0.726797385620915)

In [97]:
# Downsample the majority class
churn_majority_downsampled = resample(churn_majority, replace=False, n_samples=len(churn_minority), random_state=42)
churn_downsampled = pd.concat([churn_majority_downsampled, churn_minority])

X_downsampled = churn_downsampled[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]
y_downsampled = churn_downsampled['Churn']

# Scale and split the downsampled data
X_downsampled_scaled = scaler.fit_transform(X_downsampled)
X_train_down, X_test_down, y_train_down, y_test_down = train_test_split(X_downsampled_scaled, y_downsampled, test_size=0.3, random_state=42)

In [99]:
# Evaluate model on downsampled data
train_evaluate_model(X_train_down, y_train_down, X_test_down, y_test_down, "Downsampled")

Downsampled Data - Model Evaluation:
Accuracy: 0.7415
Precision: 0.7326
Recall: 0.7353
F1 Score: 0.7339



(0.7415329768270945,
 0.7326007326007326,
 0.7352941176470589,
 0.7339449541284404)